# Parameter estimation in GLMMs

The [`lme4`](https://github.com/lme4/lme4) package for [`R`](https://www.r-project.org) provides the `glmer` function to define and fit generalized linear mixed models (GLMMs).  The [`MixedModels`](https://github.com/dmbates/MixedModels.jl) package for [`Julia`](https://julialang.org) provides a similar function called `glmm`.  Because I am more familiar with `glmm` than with `glmer` these days, I will explain the algorithms using `glmm` for illustration.

This is a [Jupyter](https://jupyter.org) notebook of generalized linear mixed models (GLMM's) fit to data from a perception study data using the [`MixedModels`](https://github.com/dmbates/MixedModels.jl) package for [Julia](https://julialang.org).

The notebook can be run by [downloading](https://julialang.org/downloads) version 0.6.0 or later of Julia and installing the `IJulia` and `MixedModels` packages.  The Julia function to install a package is `Pkg.add()`, e.g.
```jl
Pkg.add("MixedModels")
```

When `IJulia` is installed you can start a notebook server in your browser by starting Julia in the usual way and, from within Julia, running
```jl
using IJulia
notebook()
```

Navigate to the location of this notebook file and select it to start the notebook.

An alternative is to log in to https://juliabox.com using one of the login options.  From the `File` tab, upload the data and this notebook.  Returning to the `Jupyter` tab and selecting the notebook should start it.

Once the notebook is running,  `Ctrl-Enter` runs a cell or `Shift-Enter` runs a cell and moves to the next cell.  Check under`Help -> Keyboard Shortcuts` for other key sequences.

## Making packages available

The Julia equivalent of R's `library()` function is the `using` directive.

In [1]:
using DataFrames, MixedModels, RData

Consider the `VerbAgg` data from the `lme4` package.  This is one of the data sets available in the `test/dat.rda` file for the `MixedModels` package

In [2]:
const dat = convert(Dict{Symbol,DataFrame}, load(Pkg.dir("MixedModels", "test", "dat.rda")))

Dict{Symbol,DataFrames.DataFrame} with 61 entries:
  :bs10          => 1104×6 DataFrames.DataFrame…
  :Genetics      => 60×5 DataFrames.DataFrame…
  :Contraception => 1934×6 DataFrames.DataFrame…
  :Mmmec         => 354×6 DataFrames.DataFrame…
  :kb07          => 1790×10 DataFrames.DataFrame…
  :Rail          => 18×2 DataFrames.DataFrame…
  :KKL           => 53765×24 DataFrames.DataFrame…
  :Bond          => 21×3 DataFrames.DataFrame…
  :VerbAgg       => 7584×9 DataFrames.DataFrame…
  :ergoStool     => 36×3 DataFrames.DataFrame…
  :s3bbx         => 2449×6 DataFrames.DataFrame…
  :cake          => 270×5 DataFrames.DataFrame…
  :Cultivation   => 24×4 DataFrames.DataFrame…
  :Pastes        => 60×4 DataFrames.DataFrame…
  :Exam          => 4059×5 DataFrames.DataFrame…
  :Socatt        => 1056×9 DataFrames.DataFrame…
  :WWheat        => 60×3 DataFrames.DataFrame…
  :Pixel         => 102×5 DataFrames.DataFrame…
  :Arabidopsis   => 625×8 DataFrames.DataFrame…
  :TeachingII    => 96×14 DataFra

In [3]:
const verbagg = dat[:VerbAgg]

,a,g,item,resp,id,b,s,m,r2
1,20,M,S1WantCurse,no,1,curse,other,want,N
2,11,M,S1WantCurse,no,2,curse,other,want,N
3,17,F,S1WantCurse,perhaps,3,curse,other,want,Y
4,21,F,S1WantCurse,perhaps,4,curse,other,want,Y
5,17,F,S1WantCurse,perhaps,5,curse,other,want,Y
6,21,F,S1WantCurse,yes,6,curse,other,want,Y
7,39,F,S1WantCurse,yes,7,curse,other,want,Y
8,21,F,S1WantCurse,no,8,curse,other,want,N
9,24,F,S1WantCurse,no,9,curse,other,want,N
10,16,F,S1WantCurse,yes,10,curse,other,want,Y


At present the response for a Bernoulli GLMM must be a 0/1 floating point vector so we convert the factor `r2`.

In [4]:
verbagg[:r201] = Array(float(verbagg[:r2] .== "Y"))

7584-element Array{Float64,1}:
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 ⋮  
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

## Details of evaluating the objective function

The `glmm` function only generates a `GeneralizedLinearMixedModel` object.

In [5]:
mdl = glmm(@formula(r201 ~ 1 + a + g + b + s + (1|id) + (1|item)), verbagg, Bernoulli());
typeof(mdl)

MixedModels.GeneralizedLinearMixedModel{Float64}

A separate call to `fit!` is required to fit the model.  This involves optimizing an objective function, the Laplace approximation to the deviance, with respect to the parameters, which are $\beta$, the fixed-effects coefficients, and $\theta$, the covariance parameters.  The starting estimate for $\beta$ is determined by fitting a GLM to the fixed-effects part of the formula

In [6]:
mdl.β

6-element Array{Float64,1}:
  0.206053 
  0.0399404
  0.231317 
 -0.794186 
 -1.53919  
 -0.776656 

and the starting estimate for $\theta$, which is a vector of the two standard deviations of the random effects, is chosen to be

In [7]:
mdl.θ

2-element Array{Float64,1}:
 1.0
 1.0

The Laplace approximation to the deviance requires determining the conditional modes of the random effects.  These are the values that maximize the conditional density of the random effects, given the model parameters and the data.  This is done using Penalized Iteratively Reweighted Least Squares (PIRLS).  In most cases PIRLS is fast and stable.  It is simply a penalized version of the IRLS algorithm used in fitting GLMs.

The distinction between the "fast" and "slow" algorithms in the `MixedModels` package (`nAGQ=0` or `nAGQ=1` in `lme4`) is whether the fixed-effects parameters, $\beta$, are optimized in PIRLS or in the nonlinear optimizer.

In [8]:
pirls!(mdl, true, true)

varyβ = true
obj₀ = 10210.8534389054
β = [0.206053, 0.0399404, 0.231317, -0.794186, -1.53919, -0.776656]
iter = 1
obj = 8301.483049027265
iter = 2
obj = 8205.604285133919
iter = 3
obj = 8201.896597466888
iter = 4
obj = 8201.848598910707
iter = 5
obj = 8201.848559060703


8201.848559060703

In [9]:
LaplaceDeviance(mdl)

8201.848559060703

In [10]:
mdl.β

6-element Array{Float64,1}:
  0.218535 
  0.0514385
  0.290225 
 -0.979124 
 -1.95402  
 -0.979493 

In [11]:
mdl.θ

2-element Array{Float64,1}:
 1.0
 1.0

If the optimization with respect to $\beta$ is performed within PIRLS then the nonlinear optimization of the Laplace approximation to the deviance requires optimization with respect to $\theta$ only. This is the "fast" algorithm. Given a value of $\theta$ PIRLS is used to determine the conditional estimate of $\beta$ and the conditional mode of the random effects, **b**.

In [12]:
mdl.b

2-element Array{Array{Float64,2},1}:
 [-0.600772 -1.93227 … -0.144554 -0.575224]
 [-0.186364 0.180552 … 0.282092 -0.221974] 

In [13]:
fit!(mdl, fast=true, verbose=true);

f_1: 8201.84856 [1.0, 1.0]
f_2: 8190.11782 [1.75, 1.0]
f_3: 8224.45098 [1.0, 1.75]
f_4: 9026.00391 [0.25, 1.0]
f_5: 8205.79378 [1.0, 0.25]
f_6: 8157.04103 [1.38583, 0.736457]
f_7: 8367.72422 [1.33715, 0.0]
f_8: 8170.28883 [1.41365, 1.11042]
f_9: 8158.82932 [1.27225, 0.762811]
f_10: 8161.93341 [1.40936, 0.868084]
f_11: 8156.30098 [1.32694, 0.721015]
f_12: 8156.11668 [1.32365, 0.714275]
f_13: 8156.00207 [1.31847, 0.708856]
f_14: 8155.75359 [1.32072, 0.701702]
f_15: 8155.27522 [1.32636, 0.687802]
f_16: 8154.41 [1.33859, 0.660408]
f_17: 8153.39496 [1.37582, 0.613358]
f_18: 8152.74094 [1.39515, 0.563096]
f_19: 8151.76473 [1.36763, 0.509124]
f_20: 8152.80894 [1.26776, 0.475123]
f_21: 8152.86642 [1.4148, 0.471099]
f_22: 8151.76959 [1.32589, 0.527523]
f_23: 8151.73776 [1.36681, 0.498606]
f_24: 8151.58516 [1.33974, 0.493492]
f_25: 8151.60206 [1.33758, 0.486311]
f_26: 8151.6005 [1.34692, 0.491348]
f_27: 8151.58338 [1.33958, 0.497337]
f_28: 8151.58359 [1.33927, 0.49802]
f_29: 8151.58335 [1.33972,

The optimization process is summarized by

In [14]:
mdl.LMM.optsum

Initial parameter vector: [1.0, 1.0]
Initial objective value:  8201.848559060627

Optimizer (from NLopt):   LN_BOBYQA
Lower bounds:             [0.0, 0.0]
ftol_rel:                 1.0e-12
ftol_abs:                 1.0e-8
xtol_rel:                 0.0
xtol_abs:                 [1.0e-10, 1.0e-10]
initial_step:             [0.75, 0.75]
maxfeval:                 -1

Function evaluations:     37
Final parameter vector:   [1.33956, 0.496833]
Final objective value:    8151.583340132135
Return code:              FTOL_REACHED


As one would hope, given the name, this fit is fast.

In [15]:
@time(fit!(glmm(@formula(r201 ~ 1 + a + g + b + s + (1 | id) + (1 | item)), 
        verbagg, Bernoulli()), fast=true))

  0.451232 seconds (136.88 k allocations: 16.465 MiB, 2.43% gc time)


Generalized Linear Mixed Model fit by minimizing the Laplace approximation to the deviance
  Formula: r201 ~ 1 + a + g + b + s + (1 | id) + (1 | item)
  Distribution: Distributions.Bernoulli{Float64}
  Link: GLM.LogitLink()

  Deviance (Laplace approximation): 8151.5833

Variance components:
          Column    Variance   Std.Dev. 
 id   (Intercept)  1.79443144 1.3395639
 item (Intercept)  0.24684282 0.4968328

 Number of obs: 7584; levels of grouping factors: 316, 24

Fixed-effects parameters:
              Estimate Std.Error  z value P(>|z|)
(Intercept)   0.208273  0.405425 0.513715  0.6075
a            0.0543791 0.0167533  3.24587  0.0012
g: M          0.304089  0.191223  1.59023  0.1118
b: scold       -1.0165  0.257531 -3.94708   <1e-4
b: shout       -2.0218  0.259235 -7.79912  <1e-14
s: self       -1.01344  0.210888 -4.80559   <1e-5


The alternative algorithm is to use PIRLS to find the conditional mode of the random effects, given $\beta$ and $\theta$ and then use the general nonlinear optimizer to fit with respect to both $\beta$ and $\theta$.  Because it is slower to incorporate the $\beta$ parameters in the general nonlinear optimization, the fast fit is performed and used to determine starting estimates for the more general optimization.

In [16]:
@time mdl1 = fit!(glmm(@formula(r201 ~ 1+a+g+b+s+(1|id)+(1|item)), 
        verbagg, Bernoulli()), verbose = true)

f_1: 8201.84856 [1.0, 1.0]
f_2: 8190.11782 [1.75, 1.0]
f_3: 8224.45098 [1.0, 1.75]
f_4: 9026.00391 [0.25, 1.0]
f_5: 8205.79378 [1.0, 0.25]
f_6: 8157.04103 [1.38583, 0.736457]
f_7: 8367.72422 [1.33715, 0.0]
f_8: 8170.28883 [1.41365, 1.11042]
f_9: 8158.82932 [1.27225, 0.762811]
f_10: 8161.93341 [1.40936, 0.868084]
f_11: 8156.30098 [1.32694, 0.721015]
f_12: 8156.11668 [1.32365, 0.714275]
f_13: 8156.00207 [1.31847, 0.708856]
f_14: 8155.75359 [1.32072, 0.701702]
f_15: 8155.27522 [1.32636, 0.687802]
f_16: 8154.41 [1.33859, 0.660408]
f_17: 8153.39496 [1.37582, 0.613358]
f_18: 8152.74094 [1.39515, 0.563096]
f_19: 8151.76473 [1.36763, 0.509124]
f_20: 8152.80894 [1.26776, 0.475123]
f_21: 8152.86642 [1.4148, 0.471099]
f_22: 8151.76959 [1.32589, 0.527523]
f_23: 8151.73776 [1.36681, 0.498606]
f_24: 8151.58516 [1.33974, 0.493492]
f_25: 8151.60206 [1.33758, 0.486311]
f_26: 8151.6005 [1.34692, 0.491348]
f_27: 8151.58338 [1.33958, 0.497337]
f_28: 8151.58359 [1.33927, 0.49802]
f_29: 8151.58335 [1.33972,

f_75: 8151.4109 [0.218477, 0.0564715, 0.325962, -1.0648, -2.1043, -1.06545, 1.33946, 0.491564]
f_76: 8151.40712 [0.219176, 0.0565934, 0.325719, -1.06302, -2.1028, -1.06338, 1.33951, 0.491773]
f_77: 8151.40508 [0.218683, 0.0567021, 0.324246, -1.06401, -2.1041, -1.06074, 1.3393, 0.492227]
f_78: 8151.40416 [0.217101, 0.0567959, 0.323054, -1.06822, -2.10642, -1.05795, 1.33821, 0.493367]
f_79: 8151.40597 [0.216894, 0.0568122, 0.322573, -1.07065, -2.11114, -1.05632, 1.33655, 0.493448]
f_80: 8151.40518 [0.21875, 0.0568996, 0.321267, -1.07122, -2.10867, -1.05768, 1.33799, 0.493119]
f_81: 8151.40297 [0.214685, 0.0568348, 0.322432, -1.06778, -2.10677, -1.05743, 1.33882, 0.494058]
f_82: 8151.40288 [0.212125, 0.0569334, 0.32046, -1.06926, -2.10525, -1.05573, 1.34006, 0.495352]
f_83: 8151.40429 [0.212735, 0.0570069, 0.321477, -1.06926, -2.10358, -1.05217, 1.33988, 0.495195]
f_84: 8151.40247 [0.211583, 0.0569636, 0.321333, -1.06935, -2.10837, -1.05605, 1.33938, 0.49504]
f_85: 8151.40254 [0.212001, 0

f_161: 8151.39977 [0.198799, 0.0574291, 0.320559, -1.05782, -2.10519, -1.05579, 1.34009, 0.495385]
f_162: 8151.39977 [0.198977, 0.0574256, 0.320442, -1.05801, -2.10529, -1.05579, 1.34009, 0.495416]
f_163: 8151.39977 [0.199064, 0.0574221, 0.320394, -1.05807, -2.10537, -1.05579, 1.34008, 0.495423]
f_164: 8151.39976 [0.199036, 0.0574185, 0.320301, -1.0581, -2.1053, -1.05577, 1.34004, 0.495418]
f_165: 8151.39976 [0.198886, 0.0574185, 0.320183, -1.05812, -2.10506, -1.05572, 1.33996, 0.49543]
f_166: 8151.39976 [0.198769, 0.0574126, 0.320157, -1.05797, -2.10476, -1.05572, 1.33983, 0.495493]
f_167: 8151.39975 [0.199003, 0.0574465, 0.320228, -1.05822, -2.10509, -1.05589, 1.33994, 0.495442]
f_168: 8151.39975 [0.198611, 0.0574472, 0.320195, -1.05816, -2.10511, -1.05593, 1.33988, 0.495387]
f_169: 8151.39974 [0.198519, 0.0574469, 0.320417, -1.05796, -2.10497, -1.05581, 1.33983, 0.495357]
f_170: 8151.39974 [0.198166, 0.05746, 0.320463, -1.05785, -2.10493, -1.05579, 1.33977, 0.495309]
f_171: 8151.399

Generalized Linear Mixed Model fit by minimizing the Laplace approximation to the deviance
  Formula: r201 ~ 1 + a + g + b + s + (1 | id) + (1 | item)
  Distribution: Distributions.Bernoulli{Float64}
  Link: GLM.LogitLink()

  Deviance (Laplace approximation): 8151.3997

Variance components:
          Column    Variance   Std.Dev.  
 id   (Intercept)  1.79480455 1.33970316
 item (Intercept)  0.24530373 0.49528146

 Number of obs: 7584; levels of grouping factors: 316, 24

Fixed-effects parameters:
              Estimate Std.Error  z value P(>|z|)
(Intercept)   0.199074  0.405173 0.491332  0.6232
a            0.0574279 0.0167572  3.42705  0.0006
g: M          0.320741  0.191258  1.67701  0.0935
b: scold      -1.05879  0.256794 -4.12312   <1e-4
b: shout      -2.10538  0.258518 -8.14406  <1e-15
s: self       -1.05546  0.210293   -5.019   <1e-6


This fit provided slightly better results (Laplace approximation to the deviance of 8151.400 versus 8151.583) but took 6 times as long.  That is not terribly important when the times involved are a few seconds but can be important when the fit requires many hours or days of computing time.

The comparison of the slow and fast fit is available in the optimization summary after the slow fit.

In [17]:
mdl1.LMM.optsum

Initial parameter vector: [0.208273, 0.0543791, 0.304089, -1.0165, -2.0218, -1.01344, 1.33956, 0.496833]
Initial objective value:  8151.583340132033

Optimizer (from NLopt):   LN_BOBYQA
Lower bounds:             [-Inf, -Inf, -Inf, -Inf, -Inf, -Inf, 0.0, 0.0]
ftol_rel:                 1.0e-12
ftol_abs:                 1.0e-8
xtol_rel:                 0.0
xtol_abs:                 [1.0e-10, 1.0e-10]
initial_step:             [0.405425, 0.0167533, 0.191223, 0.257531, 0.259235, 0.210888, 0.05, 0.05]
maxfeval:                 -1

Function evaluations:     205
Final parameter vector:   [0.199074, 0.0574279, 0.320741, -1.05879, -2.10538, -1.05546, 1.3397, 0.495281]
Final objective value:    8151.399719870473
Return code:              FTOL_REACHED
